 # Подготовка окружения

In [4]:
! pip install opendatasets

In [5]:
import opendatasets as od
import pandas as pd
import time
import csv
import sys
from pathlib import Path

## Скачиваем датасет

In [6]:
dataset_path = Path('new-york-city-airbnb-open-data', 'AB_NYC_2019.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: olesyamishina
Your Kaggle Key: ··········


100%|██████████| 2.44M/2.44M [00:00<00:00, 123MB/s]

In [7]:
data = pd.read_csv('new-york-city-airbnb-open-data/AB_NYC_2019.csv')
data

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


In [8]:
#среднее значение
mean = round(data.price.mean(), 2)
print(f'Среднее значение: {mean}')

Среднее значение: 152.72


In [9]:
#Дисперсия
D = round(data.price.var(), 2)
print(f'Дисперсия: {D}')

Дисперсия: 57674.03


In [50]:
#  реализуем mapper и reducer, вариант 1
#список строк файла
def get_prices(path):
    list_prices = []
    with open(path, 'r', encoding='utf8' ) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        for line in reader:
            if line[9] != 'price':
                list_prices.append(float(line[9]))
    return list_prices

#mapper
def mapper(list_price):
    return (list_price, )

#reducer - вариант 1
def reducer(price1, price2):
    if price1 is None and price2 is None:
        return None
    elif price1 is None:
        return price2
    elif price2 is None:
        return price1
    else:
        prices = []
        if len(price1) == 1:
            n, mean, M = 0, 0.0, 0.0
            prices.append(price1[0])
        else:
            n, mean, M = price1
        # в список цен добавляем очередное значения второго параметра reducer
        prices.append(price2[0])
        # проходим циклом по списку цен:
        for price in prices:
            n += 1
            delta = price - mean
            mean += delta / n
            M += delta * (price - mean)
        return n, mean, M


In [12]:
from functools import reduce

In [53]:
%%time
n, mean, D = reduce(reducer, map(mapper, get_prices('new-york-city-airbnb-open-data/AB_NYC_2019.csv')))
print(f'Среднее значение: {round(mean, 2)}, дисперсия: {round(D / (n-1), 2)}, n: {n}')
#значения сопадают со встроенными функциями

Среднее значение: 152.72, дисперсия: 57674.03, n: 48895
CPU times: user 188 ms, sys: 907 µs, total: 189 ms
Wall time: 192 ms


In [56]:

#reducer - вариант 2
def reducer(price1, price2):
    try:
        n, mean, M = price1
    except:
        n, mean, M = 0, 0.0, 0
    price = price2[0]
    if price:
        n+=1
        delta = price - mean
        mean += delta / n
        M += delta * (price - mean)
    return n, mean, M

In [57]:
%%time
n, mean, D = reduce(reducer, map(mapper, get_prices('new-york-city-airbnb-open-data/AB_NYC_2019.csv')))
print(f'Среднее значение: {round(mean, 2)}, дисперсия: {round(D / (n-1), 2)}, n: {n}')
# вариант 2 считаем немного по жругому дисперсию, хотя в прошлом ДЗ оба варианта совпали. Даже если т подставить 48895, все равно дисперсия немного отличается. Почему, интересно??

Среднее значение: 152.76, дисперсия: 57682.93, n: 48883
CPU times: user 198 ms, sys: 3.08 ms, total: 201 ms
Wall time: 201 ms


In [41]:
#обрежем датасет до id b цены
new_data =data[['id', 'price']].copy()
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      48895 non-null  int64
 1   price   48895 non-null  int64
dtypes: int64(2)
memory usage: 764.1 KB


In [ ]:
from google.colab import files
files.download(r'ny2019_price.csv')
new_data

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,id,price
0,2539,149
1,2595,225
2,3647,150
3,3831,89
4,5022,80
...,...,...
48890,36484665,70
48891,36485057,40
48892,36485431,115
48893,36485609,55
